In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import time as T
from time import time
batch_size = 64
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [ ]:
from __future__ import print_function
import keras
from keras.models import  Model
from keras.layers import Dense, Dropout, Input, Flatten, PReLU, LeakyReLU
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from time import time
import time as T
import traceback
import logging
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from GA.geneticAlgorithm import GenerationalGA
from GA.parentSelector.parentSelector import RandomParentSelector, LinealOrder, TournamentSelection
from GA.parentSelector.parentSelector import WheelSelection, LinealOrderII
from utils.datamanager import DataManager
#import tensorflow as tf

class Layer(object):
    def cross(self, other_layer):
        raise NotImplementedError

    def mutate(self):
        raise NotImplementedError

    def compare(self, other_layer):
        raise self.__repr__() == other_layer.__repr__()

    def self_copy(self):
        raise NotImplementedError

    def random_layer(self):
        raise NotImplementedError

    def __repr__(self):
        raise NotImplementedError
    
class NNLayer(Layer):
    def __init__(self, units=128, activation='relu', dropout=0):
        self.type = 'NN'
        self.posible_activations = ['relu', 'sigmoid', 'tanh', 'elu', 'prelu', 'leakyreLu']
        assert activation in self.posible_activations
        
        # parameters
        self.activation = activation
        self.dropout = dropout
        self.units = units
        
        self.units_lim = 1024
        self.units_prob = 0.2
        self.act_prob = 0.2
        self.drop_prob = 0.2

    def cross(self, other_layer):
        assert self.type == other_layer.type
        new_units = self.cross_units(other_layer.units)
        new_activation = self.cross_activation(other_layer.activation)
        new_dropout = self.cross_dropout(other_layer.dropout)
        return NNLayer(new_units, new_activation, new_dropout)

    def cross_activation(self, other_activation):
        if np.random.rand() > 0.5:
            return self.activation
        return other_activation

    def cross_dropout(self, other_dropout):
        b = np.random.rand()
        return self.dropout * (1 - b) + b * other_dropout

    def cross_units(self, other_units):
        b = np.random.rand()
        return int(self.units * (1 - b) + other_units * b)

    def mutate(self):
        aleatory = np.random.rand(4)
        if aleatory[0] < self.units_prob:
            self.units = np.random.randint(1, self.units_lim)
        if aleatory[1] < self.act_prob:
            self.activation = random.choice(self.posible_activations)
        if aleatory[2] < self.drop_prob:
            self.dropout = np.random.rand()

    '''
    def compare(self, other_layer):
        if self.units != other_layer.units:
            return False
        if self.activation != other_layer.activation:
            return False
        if self.dropout != other_layer.dropout:
            return False
        return True
    '''

    def self_copy(self):
        return NNLayer(self.units, self.activation, self.dropout)

    def random_layer(self):
        units = np.random.randint(1, self.units_lim)
        act = random.choice(self.posible_activations)
        drop = np.random.rand()
        return NNLayer(units, act, drop)

    def __repr__(self):
        return "%s|U:%d|A:%s|D:%0.3f" % (self.type, self.units, self.activation, self.dropout)
    
class CNNLayer(object):
    def __init__(self, filters=32, kernel_size=(3,3), activation='relu', dropout=0, maxpool=True):
        self.type = 'CNN'
        self.posible_activations = ['relu', 'sigmoid', 'tanh', 'elu', 'prelu', 'leakyreLu']
        assert activation in self.posible_activations
        
        # Parameters
        self.activation = activation
        self.dropout = dropout
        self.filters = filters
        self.k_size = kernel_size
        self.maxpool = maxpool
        
        self.filters_lim = 1024
        self.k_lim = 9
        self.k_prob = 0.2
        self.filter_prob = 0.2
        self.act_prob = 0.2
        self.drop_prob = 0.2
        self.maxpool_prob = 0.1

    def cross(self, other_layer):
        new_filters = self.cross_filters(other_layer.filters)
        new_ksize = self.cross_kernel(other_layer.k_size)
        new_activation = self.cross_activation(other_layer.activation)
        new_dropout = self.cross_dropout(other_layer.dropout)
        new_maxpool = self.cross_maxpool(other_layer.maxpool)
        return CNNLayer(new_filters, new_ksize, new_activation, new_dropout, new_maxpool)
    
    def cross_kernel(self, other_kernel):
        b = np.random.rand(2)
        kh = int(self.k_size[0] * b[0] + (1 - b[0]) * other_kernel[0])
        kw = int(self.k_size[1] * b[1] + (1 - b[1]) * other_kernel[1])
        return (kh, kw)
    
    def cross_maxpool(self, other_maxpool):
        return random.choice([self.maxpool, other_maxpool])

    def cross_activation(self, other_activation):
        return random.choice([self.activation, other_activation])
    
    def cross_dropout(self, other_dropout):
        b = np.random.rand()
        return self.dropout * (1 - b) + b * other_dropout

    def cross_filters(self, other_filters):
        b = np.random.rand()
        return int(self.filters * (1 - b) + other_filters * b)

    def mutate(self):
        aleatory = np.random.rand(6)
        if aleatory[0] < self.filter_prob:
            self.filters = np.random.randint(0, self.filters_lim)
        if aleatory[1] < self.act_prob:
            self.activation = random.choice(self.posible_activations)
        if aleatory[2] < self.drop_prob:
            self.dropout = np.random.rand()
        if aleatory[3] < self.k_prob:
            self.k_size = (self.k_size[0], np.random.randint(1, self.k_lim + 1))
        if aleatory[4] < self.k_prob:
            self.k_size = (np.random.randint(1, self.k_lim + 1), self.k_size[1])
        if aleatory[5] < self.maxpool_prob:
            self.maxpool = random.choice([True, False])
    '''
    def compare(self, other_layer):
        if self.filters != other_layer.filters:
            return False
        if self.activation != other_layer.activation:
            return False
        if self.dropout != other_layer.dropout:
            return False
        if self.k_size != other_layer.k_size:
            return False
        return True
    '''

    def self_copy(self):
        return CNNLayer(self.filters, self.k_size, self.activation, self.dropout, self.maxpool)

    def random_layer(self):
        filters = np.random.randint(1, self.filters_lim)
        k_size = tuple(np.random.randint(1, self.k_lim + 1, size=(2,)))
        act = random.choice(self.posible_activations)
        drop = np.random.rand()
        maxpool = random.choice([True, False])
        return CNNLayer(filters, k_size, act, drop, maxpool)

    def __repr__(self):
        return "%s|F:%d|K:(%d,%d)|A:%s|D:%0.3f|M:%d" % (self.type, self.filters, self.k_size[0], self.k_size[1],
                                         self.activation, self.dropout, self.maxpool)

In [ ]:
class Cromosome(object):

    def __init__(self, cnn_layers=[], nn_layers=[], fit=None):
        assert type(cnn_layers) == list
        assert type(nn_layers)  == list
        self.cnn_layers = cnn_layers
        self.nn_layers = nn_layers
        self.n_cnn = len(cnn_layers)
        self.n_nn  = len(nn_layers)
        
        self.max_cnn = 3
        self.max_nn = 3
        
        self.grow_prob = 0.1
        self.decrese_prob = 0.1
        self.fit = None
        self.evaluator = Fitness.get_instance()

    def set_fitness(self, fit):
        self.evaluator = fit

    def random_indiv(self):
        n_cnn = np.random.randint(0, self.max_cnn)
        n_nn = np.random.randint(0, self.max_nn)
        cnn_layers = [CNNLayer().random_layer() for i in range(n_cnn)]
        nn_layers  = [ NNLayer().random_layer() for i in range(n_nn)]
        return Cromosome(cnn_layers, nn_layers)

    @staticmethod
    def simple_indiv():
        return Cromosome([Layer()])
    
    def cross(self, other_chromosome):
        new_cnn_layers = self.cross_layers(self.cnn_layers, other_chromosome.cnn_layers)
        new_nn_layers  = self.cross_layers( self.nn_layers, other_chromosome.nn_layers)
        return Cromosome(new_cnn_layers, new_nn_layers)

    def cross_layers(self,this_layers, other_layers):
        new_layers = []

        if -len(other_layers) + 1 >= len(this_layers):
            return [l.self_copy() for l in this_layers]

        t = np.random.randint(-len(other_layers) + 1, len(this_layers))
        print(t)
        for i in range(len(this_layers)):
            j = i - t
            if j < 0 or j >= len(other_layers):
                new_layers.append(this_layers[i].self_copy())
            else:
                new_layers.append(this_layers[i].cross(other_layers[j]))
        
        return new_layers

    def mutate(self):
        self.mutate_layers(self.cnn_layers, self.max_cnn)
        self.mutate_layers(self.nn_layers, self.max_nn)
        self.n_cnn = len(self.cnn_layers)
        self.n_nn = len(self.nn_layers)
        
    ### TODO: Allow to the empty cromosomes to grow up
    def mutate_layers(self, this_layers, max_layers):
        for i in range(len(this_layers)):
            this_layers[i].mutate()
        if np.random.rand() < self.grow_prob and len(this_layers) < max_layers and len(this_layers)>0:
            this_layers.append(this_layers[0].random_layer())
        elif np.random.rand() < self.decrese_prob and len(this_layers) > 0:
            this_layers.pop()
        

    def equals(self, other_cromosome):
        return self.__repr__() == other_cromosome.__repr__()

    def __repr__(self):
        rep = ""
        for l in self.cnn_layers + self.nn_layers:
            rep += "%s\n" % (l)
        return rep

    def fitness(self, test=False):
        return self.evaluator.calc(self, test=test)
    
class Fitness:
    __instance = None

    @staticmethod
    def get_instance():
        """ Static access method. """
        if Fitness.__instance == None:
            Fitness()
        return Fitness.__instance

    def __init__(self):
        """ Virtually private constructor. """
        if Fitness.__instance != None:
            raise Exception("This class is a singleton!")
        else:
            Fitness.__instance = self

    def set_params(self, data, batch_size=128, epochs=100, early_stop=True, reduce_plateau=True, verbose=1,
                  reset=True, test=False):
        self.reset = reset
        self.test =test
        self.time = 0
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stop = early_stop
        self.reduce_plateu = reduce_plateau
        self.verb = verbose
        (self.x_train, self.y_train), (self.x_test, self.y_test), (self.x_val, self.y_val) = data
        self.num_clases = self.y_train.shape[1]
        self.callbacks = []
        if self.early_stop and keras.__version__=='2.2.4':
            #self.run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
            self.callbacks.append(EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True))
        elif self.early_stop:
            self.callbacks.append(EarlyStopping(monitor='val_acc', patience=10))
        if self.reduce_plateu:
            self.callbacks.append(ReduceLROnPlateau(monitor='val_acc', factor=0.2, 
                                                    patience=5, verbose=self.verb))
        return self

    def calc(self, chromosome, test=False):
        try:
            ti = time()
            keras.backend.clear_session()
            model = self.decode(chromosome)
            h = model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=self.epochs,
                              verbose=self.verb,
                              validation_data=(self.x_val, self.y_val),
                              callbacks=self.callbacks)
            
            if test:
                score = model.evaluate(self.x_test, self.y_test, verbose=0)
            else:
                score = model.evaluate(self.x_val, self.y_val, verbose=0)
        except Exception as e:
            score = [1 / self.num_clases, 1. / self.num_clases]
            print("Some Error with gen:")
            print(chromosome)
            logging.error(traceback.format_exc())
            keras.backend.clear_session()
            T.sleep(5)
        if self.verb and score[0] > 0:
            dataset = ['Val', 'Test'][test]
            print('%s loss: %0.4f,%s acc: %0.4f' % (dataset, score[0], dataset, score[1]))
            self.show_result(h, 'acc')
            self.show_result(h, 'loss')
        self.time += T.time() - ti
        return score[1]        
        

    def decode(self, chromosome):

        inp = Input(shape=(28, 28, 1))
        x   = Activation('linear', name='linear')(inp)
        
        for i in range(chromosome.n_cnn):
            act = chromosome.cnn_layers[i].activation
            filters = chromosome.cnn_layers[i].filters
            ksize = chromosome.cnn_layers[i].k_size
            if act in ['relu', 'sigmoid', 'tanh', 'elu']:
                x = Conv2D(filters, ksize, activation=act)(x)
            elif act == 'prelu':
                x = Conv2D(filters, ksize)(x)
                x = PReLU()(x)
            else:
                x = Conv2D(filters, ksize)(x)
                x = LeakyReLU()(x)
            x = Dropout(chromosome.cnn_layers[i].dropout)(x)
            if chromosome.cnn_layers[i].maxpool:
                x = MaxPooling2D()(x)
            
        x = Flatten()(x)
        
        for i in range(chromosome.n_nn):
            act = chromosome.nn_layers[i].activation
            if act in ['relu', 'sigmoid', 'tanh', 'elu']:
                x = Dense(chromosome.nn_layers[i].units, activation=act)(x)
            elif act == 'prelu':
                x = Dense(chromosome.nn_layers[i].units)(x)
                x = PReLU()(x)
            else:
                x = Dense(chromosome.nn_layers[i].units)(x)
                x = LeakyReLU()(x)
            x = Dropout(chromosome.nn_layers[i].dropout)(x)
        x = Dense(self.num_clases, activation='softmax')(x)

        model = Model(inputs=inp, outputs=x)
        if self.verb:
            model.summary()
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])
                      #options = self.run_opts)
        return model

    def show_result(self, history, metric='acc'):
        epochs = np.linspace(0, len(history.history['acc']) - 1, len(history.history['acc']))
        plt.plot(epochs, history.history['val_%s' % metric], label='validation')
        plt.plot(epochs, history.history[metric], label='train')
        plt.legend()
        plt.xlabel('Epochs')
        plt.ylabel(metric)
        plt.show()

    def calc_mean(self, chromosome, iters=5):
        f = []
        ti = time()
        for i in range(iters):
            f.append(self.calc(chromosome))
        print("Acc: %0.3f" % np.mean(f), np.std(f), np.max(f))
        print("Time elapsed: %0.3f" % (time() - ti))
        
    def fitness_N_models(self, c1, c2):
        from keras.layers.core import Activation
        def decode_C(chromosome, inp, name=''):
            x = Flatten()(inp)
            for i in range(chromosome.n_layers):
                act = chromosome.layers[i].activation
                if act in ['relu', 'sigmoid', 'tanh', 'elu']:
                    x = Dense(chromosome.layers[i].units, activation=act)(x)
                elif act == 'prelu':
                    x = Dense(chromosome.layers[i].units)(x)
                    x = PReLU()(x)
                else:
                    x = Dense(chromosome.layers[i].units)(x)
                    x = LeakyReLU()(x)
                x = Dropout(chromosome.layers[i].dropout)(x)
            x = Dense(self.num_clases, activation='softmax', name=name)(x)
            return x
        
        inputs = Input(shape=(28, 28, 1))
        model = Model(inputs=inputs, outputs=[decode_C(c1, inputs,'x1'), decode_C(c2, inputs,'x2')],
                        name="all_net")
        losses = {'x1':"categorical_crossentropy", 'x2':"categorical_crossentropy"}
        metrics = {'x1': 'accuracy',
                   'x2': 'accuracy'}
        model.compile(optimizer=Adam(), loss=losses, metrics=metrics)
        model.summary()
        model.fit(self.x_train, [self.y_train, self.y_train],
                  batch_size=self.batch_size,
                  epochs=self.epochs,
                  verbose=self.verb,
                  validation_data=(self.x_val, [self.y_val, self.y_val]))
                  #callbacks=self.callbacks)
        score = model.evaluate(self.x_val, [self.y_val, self.y_val], verbose=1)
        return score
        


In [ ]:

l = CNNLayer(32, (3,3), 'relu', 0, False)
l2 = CNNLayer(64, (3,3), 'relu', 0.25, True)
l3 = NNLayer(128, 'relu', 0.5)
c = Cromosome([l, l2], [l3])

ps = {'random':RandomParentSelector(), 'lineal':LinealOrder(), 'wheel':WheelSelection(), 
      'tournament':TournamentSelection(5)}

# genetic algorithm params:
parents_selector_key = 'wheel'
num_parents = 0.3
generations = 40
population = 15
train_time = 48

# Fitness params
epochs = 75
batch_size = 256
maximize_fit = True
verbose = 0
redu_plat = False
early_stop = True

# dataset params:
dataset = 'mnist'
classes = []

p = ps[parents_selector_key]

# Load data
dm = DataManager(dataset, clases=classes)
data = dm.load_data()
fitness = Fitness.get_instance()
fitness.set_params(data, verbose=verbose, reduce_plateau=redu_plat, 
                   epochs=epochs, early_stop=early_stop)


In [ ]:
ti_all = time()
generational = GenerationalGA(num_parents=num_parents, chromosome=c, parent_selector=p, generations=generations,
                              num_population=population, crossover_prob=0.5, mutation_prob=0.7,
                              maximize_fitness=maximize_fit, training_hours=train_time)
winner, best_fit, ranking = generational.evolve()
print("Total elapsed time: %0.3f" % (time() - ti_all))
print("Total training time: %0.3f" % fitness.time)